In [1]:
print("Ok")

Ok


In [2]:
%pwd

'e:\\AI-Driven-Medical-Chatbot-\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'e:\\AI-Driven-Medical-Chatbot-'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [8]:
extracted_data=load_pdf_file(data='Data/')

In [9]:
# extracted_data

In [10]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [12]:
# text_chunks

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [14]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [16]:
embeddings = download_hugging_face_embeddings()

In [17]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [18]:
# query_result

In [33]:
from dotenv import load_dotenv
load_dotenv()

True

In [70]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GEMINI_API_KEY = os.environ.get('GEMINI_API_KEY')


In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [75]:
import os
import google.generativeai as genai
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
GEMINI_API_KEY = os.environ.get('GEMINI_API_KEY')

In [ ]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [41]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

Error while installing plugin inference: can't set attribute 'inference'
Traceback (most recent call last):
  File "d:\anaconda3\envs\medibot\lib\site-packages\pinecone_plugin_interface\actions\installation.py", line 13, in install_plugins
    setattr(target, plugin.namespace, impl(target.config, plugin_client_builder))
AttributeError: can't set attribute 'inference'


In [76]:
docsearch

In [77]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [78]:
retrieved_docs = retriever.invoke("What is Acne?")

In [79]:
retrieved_docs

[Document(id='f9596e99-99b4-4313-af65-8d48092a66ad', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='3ce11d0c-d503-4259-9643-07c5b9167a3c', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM -

In [82]:
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.4, max_tokens=500)

In [83]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [84]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [85]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly and gigantism are disorders caused by abnormal growth hormone release from the pituitary gland.  Gigantism occurs when this abnormality happens before bone growth stops, resulting in unusual height.  Acromegaly occurs after bone growth stops, causing increased bone and soft tissue growth, along with other bodily disturbances.


In [86]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

This question cannot be answered from the given context. The provided text discusses blood counts and athletic heart syndrome, but does not define or explain "stats".  More information is needed to answer your question.
